In [2]:
import pandas as pd
import numpy as np
import pickle
import spacy
import regex as re
from spacy.lang.de import German
from spacy.matcher import Matcher
from spacy.tokens import Span, Token, Doc
from langdetect import detect, detect_langs
from geopy.geocoders import Nominatim

# Load pickled df_lgbt

In [3]:
with open('pickles/df_lgbt.pickle', 'rb') as f:
    df_lgbt = pickle.load(f)

In [3]:
stories = list(df_lgbt['Story'])
titles = list(df_lgbt['Header'])
districts = list(df_lgbt['District'])
sources = list(df_lgbt['Source'])
dates = list(df_lgbt['Date'])
indices = list(df_lgbt.index) # indices of incidents in df_complete

# Load pickled df_map

In [3]:
with open('pickles/df_map_test.pickle', 'rb') as f:
    df_map = pickle.load(f)

# Load pickled docs

In [5]:
with open('pickles/spacy_docs.pickle', 'rb') as f:
    docs = pickle.load(f)

### Detect languages

All stories are in German. But this led me to find that there are 5 missing stories.

In [226]:
languages_stories = []
missing_stories = [] # index of missing stories in LIST stories
for i, story in enumerate(stories):
    try:
        lan = detect(story)
    except:
        lan = np.nan
        missing_stories.append(i)
    languages_stories.append(lan)

In [243]:
# only German. Some missing
set(languages_stories)

{'de', nan}

### Find missing stories

In [241]:
story_nan_indices = []
for i, lan in enumerate(languages_stories):
    if type(lan) != str:
        story_nan_indices.append(i)
story_nan_indices

[11, 20, 25, 98, 109]

In [244]:
# info on incidents without story
no_story_info = [(i, str(dates[i].date()), districts[i], titles[i], sources[i]) for i in story_nan_indices]
no_story_info

[(11,
  '2014-08-11',
  'Friedrichshain-Kreuzberg',
  'Transphobe Beleidigung',
  'Register Friedrichshain-Kreuzberg'),
 (20,
  '2014-06-14',
  'Friedrichshain-Kreuzberg',
  'Transphobe Beleidigung',
  'Register Friedrichshain-Kreuzberg'),
 (25,
  '2014-05-18',
  'Friedrichshain-Kreuzberg',
  'Mann aus homophoben Gründen zu Boden geschlagen',
  'ReachOut'),
 (98,
  '2015-04-14',
  'Lichtenberg',
  'Transphober Angriff in der Tram',
  'Lichtenberger Register'),
 (109,
  '2015-03-05',
  'Friedrichshain-Kreuzberg',
  'Homophobe Beleidigung',
  'Register Friedrichshain-Kreuzberg')]

In [240]:
# indices in df_lgbt['Stories']
missing_stories_indices = [df_lgbt.iloc[i].name for i in missing_stories]
missing_stories_indices

[2063, 2261, 2350, 4310, 4469]

In [71]:
languages_titles = []
for title in titles:
    try:
        lan = detect(title)
    except:
        lan = np.nan
    languages_titles.append(lan)

In [75]:
title_f_lan_indices = []
title_f_lan_lan = []
for i, lan in enumerate(languages_titles):
    if lan != 'de':
        title_f_lan_indices.append(i)
        title_f_lan_lan.append(lan)

In [81]:
foreign_titles = [titles[i] for i in title_f_lan_indices]

In [82]:
foreign_titles

['Angriff aus transphober Motivation',
 'Angriff aus transphober Motivation',
 'Angriff aus transphober Motivation',
 'Facebook-Seite der NPD Pankow',
 'Angriff aus homophober Motivation',
 'Angriff aus homophober Motivation',
 'Homepage der NPD KV8 Pankow',
 'Homophobie',
 'Angriff aus homophober Motivation in der Motzstr. ',
 'Angriff aus homophober Motivation',
 'Homophob motivierter Angriff',
 'Homophob motivierter Angriff',
 'Homophobie',
 'Angriff aus homophober Motivation',
 'Homophobie',
 'Angriff aus homophober Motivation',
 'Homophober Angriff am S-Bahnhof Tempelhof',
 'Homophober Angriff in Rudow',
 'Transhass in Charlottenburg',
 'Homophobe Propaganda',
 'Transsexuelle(n) attackiert',
 'Homophob motivierter Angriff',
 'Homophob angegriffen',
 'Transhass in Charlottenburg',
 'Homophob motivierter Angriff',
 'Mann aus homophober Motivation getreten',
 'Facebook-Dokumentation Oktober: "AfD Treptow-Köpenick" und „AfD-Fraktion Treptow-Köpenick“',
 'Facebook-Dokumentation Oktober

In [ ]:
# This seems redundant now...
"""
locations = []
organizations = []
for ent in doc.ents:
    if ent.label_ == 'LOC':
        locations.append(ent.text)
    elif ent.label_ == 'ORG':
        organizations.append(ent.text)
locations, organizations
"""

## Find locations

**Find coordinates**

In [284]:
docs[83]

(LGBTIQ*feindliche Beleidigung und Angriff vor der Volksbühne. Mehrere Männer und Frauen saßen auf den Stufen vor der Volksbühne am Rosa-Luxemburg-Platz, als gegen 0.15 Uhr ein Mann mit einem Fahrrad vorbeikam, die Gruppe bepöbelte und dabei einen 20 Jahre alten Mann homophob beleidigte. Der Unbekannte entfernte sich zunächst kurz, kam dann aber etwa eine viertel Stunde später zurück und versprühte Reizstoff auf die Gruppe. Zwei Männer, 21 und 33 Jahre alt, erlitten dabei Augen- und Atemwegsreizungen. Der Angreifer flüchtete.,
 {'index': 18525,
  'district': 'Mitte',
  'MISC': ['Angriff vor der Volksbühne'],
  'LOC': ['Volksbühne am Rosa-Luxemburg-Platz']})

In [33]:
list_of_entities = [doc[0].ents for doc in docs]
list_of_districts = list(df_lgbt['District'])

In [34]:
# test number of locations found
i=10
#len(location_finder(list_of_entities[i], list_of_districts[i])[0])
location_finder(list_of_entities[i], list_of_districts[i])[0]

[{'place_id': 256795914,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 408306,
  'boundingbox': ['52.6196447', '52.6755087', '13.4200259', '13.523022'],
  'lat': '52.6366724',
  'lon': '13.4999292',
  'display_name': 'Buch, Pankow, Berlin, Deutschland',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.7639251505982987,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png',
  'address': {'suburb': 'Buch',
   'borough': 'Pankow',
   'state': 'Berlin',
   'country': 'Deutschland',
   'country_code': 'de'}},
 {'place_id': 96502140,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 29319916,
  'boundingbox': ['52.5286422', '52.5289603', '13.4372366', '13.4380423'],
  'lat': '52.5287836',
  'lon': '13.43764116835994',
  'display_name': 'Denkmal des polnischen Soldaten und deutschen 

In [36]:
docs[10][0].ents

(NPD Pankow,
 NPD,
 Buch,
 deutschen,
 Buch,
 NPD,
 Wagenburg,
 Karow,
 Punker_innen und Linke,
 NPD,
 Deutschen)

In [40]:
# find how well spacy detects locations
# This took +1 hour to run. I think it's because bad_ents_lol. They're Span obejcts
good_loc_lol = []
bad_loc_lol = []
bad_ents_lol = []

for doc in docs:
    ents = doc[0].ents
    district = doc[1]['district']
    good_loc_list, bad_loc_list, bad_ents = location_finder(ents, district)
    good_loc_lol.append(good_loc_list)
    bad_loc_lol.append(bad_loc_list)
    bad_ents_lol.append(bad_ents)

**Pickle initial locations found by location_finder()**

In [63]:
# pickle
with open('pickles/location_finder_good_bad_tuple.pickle', 'wb') as f:
    pickle.dump((good_loc_lol, bad_loc_lol), f)

In [13]:
# load
with open('pickles/location_finder_good_bad_tuple.pickle', 'rb') as f:
    good, bad = pickle.load(f)

In [14]:
len([i for i, loc_list in enumerate(good) if len(loc_list) == 0])

141

## Find labels

In [273]:
df_map[df_map['Structural Discrimination'].isna() == False]

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status
4405,2015-03-23,Treptow-Köpenick,Homophobe Vermietungspolitik,Zwei Frauen versuchten bei einer privaten Gart...,Zentrum für Demokratie,None,NaN,NaN,None,yes,None,None


In [267]:
df_map

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status,date_form,icon_data
1707,2014-11-29,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Ein 30-jähriger Mann wird gegen 05.00 Uhr in d...,ReachOut,physical,52.499447,13.359882,NaN,NaN,NaN,NaN,29 Nov 2014,test
1836,2014-10-26,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Am 26.10.14 wird ein 30-jähriger Mann gegen 07...,ReachOut,physical,52.494660,13.361087,NaN,NaN,NaN,NaN,26 Oct 2014,test
1915,2014-10-03,Mitte,Homophobe Aggression gegen ein schwules Pärchen,Zwei Schwule werden auf dem U-Bahnhof Pankstra...,Polizei Berlin,physical,52.550920,13.384846,NaN,NaN,NaN,NaN,03 Oct 2014,test
1940,2014-09-24,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Auf der Potsdamer Straße ist gegen 11.20 Uhr e...,ReachOut,physical,52.494660,13.361087,NaN,NaN,NaN,NaN,24 Sep 2014,test
1951,2014-09-21,Pankow,Homophober Angriff am S-Bahnhof Blankenburg,Zwei 20- und 21-jährige Männer werden gegen 00...,ReachOut,physical,52.591013,13.442641,NaN,NaN,NaN,NaN,21 Sep 2014,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19354,2020-05-31,Neukölln,LGBTIQ*-feindlicher Angriff in Nordneukölln,Am Sonntagabend wurden zwei Personen in der Fl...,"Register Neukölln, rbb24",physical,52.480463,13.422389,NaN,NaN,NaN,NaN,31 May 2020,test
19591,2020-05-11,Tempelhof-Schöneberg,LGBTIQ*feindliche Angriff,Gegen 0.30 Uhr fährt eine 25-jährige Frau mit ...,ReachOut Berlin,physical,52.500700,13.347684,NaN,NaN,NaN,NaN,11 May 2020,test
19700,2020-05-01,Neukölln,LGBTIQ*-feindlicher Angriff in Nordneukölln,Ein 33-jähriger Mann wird gegen 15.45 Uhr in d...,ReachOut Berlin,physical,52.463716,13.433574,NaN,NaN,NaN,NaN,01 May 2020,test
20279,2020-02-22,Friedrichshain-Kreuzberg,Frau am Mehringplatz aus LGBTIQ*-Feindlichkeit...,Um 21 Uhr soll eine 28-jährige Transfrau am Me...,ReachOut Berlin; https://www.berlin.de/polizei...,physical,52.498905,13.391777,NaN,NaN,NaN,NaN,22 Feb 2020,test


In [11]:
len(df_lgbt)

1006

In [239]:
[doc for doc in docs if doc[0]._.index in [11687, 11773, 15566, 18842]]

[(Israel-bezogener Antisemitismus auf Aufkleber. An einer Ampel auf einer Verkehrsinsel in der Neuköllner Flughafenstraße wird ein Aufkleber mit der Aufschrift "israel is an apartheid state", inklusive  hebräischer Übersetzung und "#Queers against Occupation" entdeckt.,
  {'index': 11687,
   'district': 'Neukölln',
   'PER': ['Israel-bezogener'],
   'LOC': ['Neuköllner Flughafenstraße']}),
 (Anti-Israel Propaganda beim lesbisch-schwulen "Motzstraßenfest". Rund um das Motzstraßenfest werden am 21.07.2018 Sticker gefunden, auf denen "No Pride in Israeli Apartheid" zu lesen ist. Außerdem "Free Palestine.Tear down this wall! End Israeli Occuptaion".,
  {'index': 11773,
   'district': 'Tempelhof-Schöneberg',
   'PER': ['Anti-Israel Propaganda',
    'Motzstraßenfest',
    'End Israeli Occuptaion'],
   'LOC': ['Motzstraßenfest'],
   'MISC': ['No Pride', 'Free Palestine', 'Tear down this']}),
 (Homofeindliche Schmiererei in Mitte. Im Ortsteil Mitte wurde in der Alexanderstraße (Höhe Hausnummer

In [249]:
df_lgbt.loc[18842, 'Attack'] = 'physical'

In [251]:
df_map[df_map['Attack'] == 'check']

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status


In [250]:
df_map['Attack'].value_counts()

physical    390
verbal      251
Name: Attack, dtype: int64

In [214]:
df_lgbt.loc[[1861,2063,2194,4405]]

,Date,District,Header,Story,Source,Attack,Propaganda,Structural Discrimination,Material Damage,Reported status
1861,2014-10-17,Mitte,LGBQT feindliche Schmiererei entdeckt!,Am Abend gegen 20.10 Uhr stellen Polizeibeamte...,Polizei Berlin,None,anonymous,None,yes,yes
2063,2014-08-11,Friedrichshain-Kreuzberg,Transphobe Beleidigung,,Register Friedrichshain-Kreuzberg,verbal,None,None,None,None
2194,2014-07-04,Mitte,Homophober Angriff U-Bahnhof Französische Strasse,Gegen 21.00 Uhr werden ein 21-jähriger und ein...,ReachOut,physical,None,None,None,None
4405,2015-03-23,Treptow-Köpenick,Homophobe Vermietungspolitik,Zwei Frauen versuchten bei einer privaten Gart...,Zentrum für Demokratie,None,None,yes,None,None


In [222]:
df_map.loc[[1861,2063,2194,4405]]

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status
1861,2014-10-17,Mitte,LGBQT feindliche Schmiererei entdeckt!,Am Abend gegen 20.10 Uhr stellen Polizeibeamte...,Polizei Berlin,None,52.513258,13.376137,anonymous,None,yes,yes
2063,2014-08-11,Friedrichshain-Kreuzberg,Transphobe Beleidigung,,Register Friedrichshain-Kreuzberg,verbal,NaN,NaN,None,None,None,None
2194,2014-07-04,Mitte,Homophober Angriff U-Bahnhof Französische Strasse,Gegen 21.00 Uhr werden ein 21-jähriger und ein...,ReachOut,physical,52.514400,13.389158,None,None,None,None
4405,2015-03-23,Treptow-Köpenick,Homophobe Vermietungspolitik,Zwei Frauen versuchten bei einer privaten Gart...,Zentrum für Demokratie,None,NaN,NaN,None,yes,None,None


In [221]:
for k,v in dict_no_ents.items():
    try:
        df_map.loc[k, 'Reported status'] = v['reported_police']
    except:
        continue

In [ ]:
dict_no_ents[1861]

In [180]:
dict_no_ents

{1861: {'location': 'Denkmal für die im Nationalsozialismus verfolgten Homosexuellen',
  'attack': None,
  'propaganda': 'anonymous',
  'damage_of_property': 'yes',
  'structural_discrimination': None,
  'reported_police': 'yes'},
 2063: {'location': None,
  'attack': 'verbal',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2194: {'location': 'metro Französische Straße',
  'attack': 'physical',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2261: {'location': None,
  'attack': 'verbal',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2275: {'location': 'metro Französische Straße',
  'attack': 'physical',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2511: {'location': 'metro Französische Straße',
  'attack'

## Label stories without entities

In [23]:
dict_no_ents

{1861: {'location': 'Denkmal für die im Nationalsozialismus verfolgten Homosexuellen',
  'attack': None,
  'propaganda': 'anonymous',
  'damage_of_property': 'yes',
  'structural_discrimination': None,
  'reported_police': 'yes'},
 2063: {'location': None,
  'attack': 'verbal',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2194: {'location': 'metro Französische Straße',
  'attack': 'physical',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2261: {'location': None,
  'attack': 'verbal',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2275: {'location': 'metro Französische Straße',
  'attack': 'physical',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2511: {'location': 'metro Französische Straße',
  'attack'

In [6]:
# load pickled dict_no_ents:
with open('pickles/dict_no_ents.pickle', 'rb') as f:
    dict_no_ents = pickle.load(f)

In [37]:
# local indices of 1009 stories
local_indices = [i for i, loc_list in enumerate(good) if len(loc_list) == 0]

In [38]:
# docs without entities
docs_no_ents = [docs[i] for i in local_indices]

In [342]:
physical = 'physical'
verbal = 'verbal'
anonymous = 'anonymous'
identified = 'identified'
yes = 'yes'

In [42]:
docs_no_ents

[(LGBQT feindliche Schmiererei entdeckt!. Am Abend gegen 20.10 Uhr stellen Polizeibeamte eine Farbschmiererei an dem Denkmal für die im Nationalsozialismus verfolgten Homosexuellen fest.,
  {'index': 1861, 'district': 'Mitte', 'MISC': ['LGBQT']}),
 (Transphobe Beleidigung. ,
  {'index': 2063,
   'district': 'Friedrichshain-Kreuzberg',
   'PER': ['Transphobe']}),
 (Homophober Angriff U-Bahnhof Französische Strasse. Gegen 21.00 Uhr werden ein 21-jähriger und ein 32-jähriger Mann auf demBahnsteig des U-Bahnhofs Französische Straße von drei Männern im Altervon 17 bis 19 Jahren homophob beleidigt, geschlagen und verletzt.,
  {'index': 2194,
   'district': 'Mitte',
   'LOC': ['U-Bahnhofs', 'Französische Straße von drei Männern']}),
 (Transphobe Beleidigung. ,
  {'index': 2261,
   'district': 'Friedrichshain-Kreuzberg',
   'PER': ['Transphobe']}),
 (Homophobe Aggression und Beleidigung auf U-Bahnhof Französische Straße. Gegen 1.00 Uhr wird ein 44-jähriger Mann auf dem U-Bahnhof Französische S

TODO:
* Think what to do with Nordneukölln
* update reported_police info after seing source
* check google for cases where corners are given str. ecke str

In [40]:
docs_no_ents[44][0]._.district

'Spandau'

In [578]:
i += 1 # change
i, docs_no_ents[i][0]._.district, docs_no_ents[i][0]

(50,
 'Lichtenberg',
 Flüchtlingsfeindliche Beiträge auf NPD-Facebookseite. Die Lichtenberger NPD veröffentlichte auf Facebook hauptsächlich Beiträge, die sich gegen Flüchtlinge richteten. So wurden im Umfeld von Unterkünften NPD-Aufkleber fotografiert und gepostet. Verfolgten homosexuellen Flüchtlingen aus Tschetschenien wurde unterstellt, dass sie betrügen würden, um später ihre Familien nachholen zu können.)

In [579]:
dict_no_ents[docs_no_ents[i][0]._.index] = {
    'location': None, 
    'attack': None,                    # 'physical' | 'verbal' | None
    'propaganda': identified,                # 'anonymous' | 'identified' | None
    'damage_of_property': None,        # 'yes' | None
    'structural_discrimination': None, # 'yes' | None
    'reported_police': None                  # 'yes' | None
}

#pickle
with open('pickles/dict_no_ents.pickle', 'wb') as f:
    pickle.dump(dict_no_ents, f)

len(dict_no_ents)

51

In [576]:
Nominatim(user_agent="mymap").geocode('Michaelkirchplatz Berlin', addressdetails=True).raw

{'place_id': 223100945,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 659300598,
 'boundingbox': ['52.5062536', '52.5079856', '13.4180292', '13.4204636'],
 'lat': '52.5071228',
 'lon': '13.419247474125797',
 'display_name': 'Michaelkirchplatz, Luisenstadt, Mitte, Berlin, 10179, Deutschland',
 'class': 'leisure',
 'type': 'park',
 'importance': 0.37000000000000005,
 'address': {'leisure': 'Michaelkirchplatz',
  'neighbourhood': 'Luisenstadt',
  'suburb': 'Mitte',
  'borough': 'Mitte',
  'city': 'Berlin',
  'state': 'Berlin',
  'postcode': '10179',
  'country': 'Deutschland',
  'country_code': 'de'}}

In [505]:
dict_no_ents#[2511]['location'] = 'metro Französische Straße'

{1861: {'location': 'Denkmal für die im Nationalsozialismus verfolgten Homosexuellen',
  'attack': None,
  'propaganda': 'anonymous',
  'damage_of_property': 'yes',
  'structural_discrimination': None,
  'reported_police': 'yes'},
 2063: {'location': None,
  'attack': 'verbal',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2194: {'location': 'metro Französische Straße',
  'attack': 'physical',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2261: {'location': None,
  'attack': 'verbal',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2275: {'location': 'metro Französische Straße',
  'attack': 'physical',
  'propaganda': None,
  'damage_of_property': None,
  'structural_discrimination': None,
  'reported_police': None},
 2511: {'location': 'metro Französische Straße',
  'attack'

**Find locations for missing entries after labeling 51 first ones by hand**

In [ ]:
'''
'heteronormativ|gay|lgbt|lgtb|lbgt|ltgb|lgbqt|schwul|schwuchtel|lsbt|transgender|'\
'transphob|transsex|transfrau|transperson|transmann|transfeind|homophob|queer|gleichgeschlecht|'\
'homosexu|homofeindlich|sexuelle[rn]* [ovi]|[^a-zöäüß]gender|binär'
'''

In [104]:
len(docs_no_ents[51:])

# parse texts and look for regex
loc_lexicon = '[-a-zA-Züöäß]+ ?str[ .,a][sße]{0,3}|[-a-zA-Züöäß]+ ?allee|'\
'[-a-zA-Züöäß]+ ?damm[ .,]+|[-a-zA-Züöäß]+ ?weg[ .,]+|[-a-zA-Züöäß]+ ?platz[ .,]+'

street_matches = {}
for doc in docs_no_ents[51:]:
    doc_i = doc[0]._.index
    district = doc[0]._.district
    story = doc[0].text
    matches = re.findall(loc_lexicon, doc[0].text, flags=re.IGNORECASE)
    for match in matches:
        query = match + ' ' + district + ' Berlin'
        try:
            loc = Nominatim(user_agent="mymap").geocode(query, addressdetails=True).raw
            if loc['address']['borough'] == district:
                street_matches[doc_i] = match
        except:
            continue


#df_lgbt = df_2014_2021[(df_2014_2021['Story'].str.contains(lgbt_lexicon, flags=re.IGNORECASE) == True)|(df_2014_2021['Header'].str.contains(lgbt_lexicon, flags=re.IGNORECASE) == True)]


In [133]:
[(doc[0]._.district, doc[0]) for doc in docs if doc[0]._.index in desambiguate_i][6]

('Neukölln',
 LGBTIQ*-feindlicher Angriff in Nordneukölln. Ein 33-jähriger Mann wird gegen 15.45 Uhr in der Hermannstraße von einem unbekannten Mann aus LGBTIQ*feindlicher Motivation beleidigt und auf die Straße gestoßen. Der Unbekannte versucht das Handy des 33-Jährigen zu rauben.)

In [159]:
street_matches[9091]

'Mehringdamm '

In [160]:
for k, v in street_matches.items():
    dict_no_ents[k] = {'location': v}

In [177]:
with open('pickles/dict_no_ents.pickle', 'wb') as f:
    pickle.dump(dict_no_ents, f)

In [130]:

Nominatim(user_agent="mymap").geocode('Straße An der Urania Tempelhof-Schöneberg berlin', addressdetails=True).raw

{'place_id': 18609173,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 1927273858,
 'boundingbox': ['52.5006496', '52.5007496', '13.3476336', '13.3477336'],
 'lat': '52.5006996',
 'lon': '13.3476836',
 'display_name': 'An der Urania, Kleiststraße, Schöneberg, Tempelhof-Schöneberg, Berlin, Deutschland',
 'class': 'highway',
 'type': 'bus_stop',
 'importance': 0.711,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//transport_bus_stop2.p.20.png',
 'address': {'highway': 'An der Urania',
  'road': 'Kleiststraße',
  'suburb': 'Schöneberg',
  'borough': 'Tempelhof-Schöneberg',
  'city': 'Berlin',
  'state': 'Berlin',
  'country': 'Deutschland',
  'country_code': 'de'}}

In [99]:
docs_no_ents[-8]

(Diskriminierung am Arbeitsplatz im Nollendorfkiez . Im Nollendorfkiez wurde eine Person über längeren Zeitraum am Arbeitsplatz diskriminiert: rassistische, ablelistische und homophobe Kommentaren wurden von Vorgesetzten getätigt.,
 {'index': 20214,
  'district': 'Tempelhof-Schöneberg',
  'MISC': ['Diskriminierung am Arbeitsplatz'],
  'LOC': ['Nollendorfkiez', 'Nollendorfkiez']})

In [94]:
street_matches

{9091: ['Mehringdamm '],
 9212: [],
 9272: [],
 9440: [],
 9706: [],
 9959: [],
 10294: ['Weserstraße'],
 10305: ['drei Straße'],
 10338: [],
 10352: [],
 10410: ['Müllerstraße'],
 10822: [],
 10939: ['Festplatz ', 'Kurt-Schuhmacher-Damm '],
 11041: ['Gerichtsstraße'],
 11303: ['Hamburger Straße'],
 11471: ['von Stra'],
 11646: [],
 11687: ['Flughafenstraße'],
 11773: ['Motzstraße', 'Motzstraße'],
 11789: [],
 11882: ['Prinzenstraße'],
 12330: [],
 12365: ['Müllerstraße'],
 13143: [],
 13235: [],
 13425: ['Hermannplatz. '],
 13788: [],
 13844: ['Urbanstraße', 'Urbanstraße'],
 13892: [],
 13900: [],
 13917: [],
 13948: [],
 13969: [],
 14128: [],
 14214: ['Narkauer Weg '],
 14278: [],
 14563: ['Gneisenaustraße', 'Gneisenaustraße'],
 14650: [],
 14788: [],
 14948: [],
 14999: [],
 15066: [],
 15240: ['Kottbusser Damm '],
 15242: ['Fischerhüttenstraße'],
 15333: ['Baumschulenweg. '],
 15411: [],
 15437: [],
 15566: ['Alexanderstraße'],
 15648: [],
 16034: [],
 16035: ['Nollendorfplatz '],

In [60]:
loc_lexicon

'[-a-zA-Züöäß]+ ?str[ .a][sße]{0,3}\x08|[-a-zA-Züöäß]+ ?allee\x08|[-a-zA-Züöäß]+ ?damm\x08|[-a-zA-Züöäß]+ ?weg\x08|\\[-a-zA-Züöäß]+ ?platz\x08'

In [72]:
loc_lexicon = '[-a-zA-Züöäß]+ ?str[ .,a][sße]{0,3}|[-a-zA-Züöäß]+ ?allee|[-a-zA-Züöäß]+ ?damm[ .,]+|[-a-zA-Züöäß]+ ?weg[ .,]+|[-a-zA-Züöäß]+ ?platz[ .,]+'

In [73]:
loc_lexicon

'[-a-zA-Züöäß]+ ?str[ .,a][sße]{0,3}|[-a-zA-Züöäß]+ ?allee|[-a-zA-Züöäß]+ ?damm[ .,]+|[-a-zA-Züöäß]+ ?weg[ .,]+|[-a-zA-Züöäß]+ ?platz[ .,]+'

In [79]:
re.findall(loc_lexicon, docs_no_ents[51][0].text, flags=re.IGNORECASE)

['Mehringdamm ']

In [88]:
docs_no_ents[51][0].text

'Homophob beleidigt und bespuckt. Vergangene Nacht wurde die Polizei wegen eines homophoben Übergriffs zum U-Bahnhof Mehringdamm gerufen. Dort gab der 20-jährige Geschädigte an, aus einer Gruppe heraus von einem 18-Jährigen aufgrund seiner Frauenkleider beleidigt und bespuckt worden zu sein. Bei der anschließenden Personalienfeststellung des Tatverdächtigen ließ dieser ein Tütchen mit Drogen fallen, die von den Beamten gesichert wurden.\nNr. 1299'

In [43]:
len(dict_no_ents)

82

**Get coordinates of found locations**

In [125]:
# improve this to carry the original index
zero_loc_indices = []
one_loc_locs = []
one_loc_indices = []
two_loc_locs = []
two_loc_indices = []
three_loc_locs = []
three_loc_indices = []
for i, loc in enumerate(good_loc_lol):
    if len(loc) == 1:
        one_loc_indices.append(i)
        one_loc_locs.append((loc[0]['lat'], loc[0]['lon']))
    elif len(loc) == 0:
        zero_loc_indices.append(i)
    elif len(loc) == 2:
        two_loc_indices.append(i)
        two_loc_locs.append(((loc[0]['lat'], loc[0]['lon']), 
                             (loc[1]['lat'], loc[1]['lon'])))
    elif len(loc) == 3:
        three_loc_indices.append(i)
        three_loc_locs.append(((loc[0]['lat'], loc[0]['lon']), 
                               (loc[1]['lat'], loc[1]['lon']), 
                               (loc[2]['lat'], loc[2]['lon'])))
        

In [126]:
# zero_loc_indices are the files I will label
len(one_loc_indices), len(zero_loc_indices), len(two_loc_indices), len(three_loc_indices)

(253, 141, 283, 182)

**Locations with one pair of coordinates**

In [206]:
# find REAL indices
one_loc_indices_real = [docs[i][1]['index'] for i in one_loc_indices]
zero_loc_indices_real = [docs[i][1]['index'] for i in zero_loc_indices]
two_loc_indices_real = [docs[i][1]['index'] for i in two_loc_indices]
three_loc_indices_real = [docs[i][1]['index'] for i in three_loc_indices]

In [286]:
latitude = {i:float(lat[0]) for (i, lat) in zip(one_loc_indices_real, one_loc_locs)}
longitude = {i:float(lon[1]) for (i, lon) in zip(one_loc_indices_real, one_loc_locs)}

In [288]:
df_lgbt['Latitude'] = df_lgbt.index.map(latitude)
df_lgbt['Longitude'] = df_lgbt.index.map(longitude)

**Check loc entities that are suggesting two places**
<br>
- managed to retrieve 97 doubled locations in the cases where 2 were detected

In [290]:
# here I realized that some places can just be repeated 
two_loc_indices_real[0]
[doc[0].ents for doc in docs if doc[0]._.index==1951]

[(Homophober Angriff, S-Bahnhof Blankenburg, S-Bahnhof Blankenburg)]

In [291]:
# These pairs of coordinates are identical
two_to_one_loc_dict = {i:a for i, (a, b) in zip(two_loc_indices_real, two_loc_locs) if a == b}

In [308]:
for k, v in two_to_one_loc_dict.items():
    df_lgbt.loc[k, 'Latitude'] = float(v[0])
    df_lgbt.loc[k, 'Longitude'] = float(v[1])

In [309]:
df_lgbt[df_lgbt['Latitude'].isna() == False].shape

(350, 12)

In [295]:
df_lgbt[df_lgbt['Latitude'].isna() == False].shape

(253, 12)

In [312]:
df_map_test = df_lgbt.drop(['Propaganda', 'Structural Discrimination', 'Material Damage', 'Reported status'], axis=1)


In [317]:
df_map_test.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'}, inplace=True)

**Pickle df_map_test**

In [318]:
# pickle
with open('df_map_test.pickle', 'wb') as f:
    pickle.dump(df_map_test, f)

In [371]:
Nominatim(user_agent="mymap").geocode('Berlin', addressdetails=True).raw

{'place_id': 574401,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 240109189,
 'boundingbox': ['52.3570365', '52.6770365', '13.2288599', '13.5488599'],
 'lat': '52.5170365',
 'lon': '13.3888599',
 'display_name': 'Berlin, 10117, Deutschland',
 'class': 'place',
 'type': 'city',
 'importance': 0.8975390282491362,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_place_city.p.20.png',
 'address': {'city': 'Berlin',
  'state': 'Berlin',
  'postcode': '10117',
  'country': 'Deutschland',
  'country_code': 'de'}}

In [350]:
df_map_test['Date'].iloc[0].strftime('%d %d %Y')

'29 29 2014'

In [357]:
df_map_test['Date'].apply(lambda d: d.strftime('%d %d %Y'))

1707     29 29 2014
1836     26 26 2014
1861     17 17 2014
1874     12 12 2014
1913     03 03 2014
            ...    
21900    14 14 2021
21930    10 10 2021
21933    10 10 2021
21948    07 07 2021
21974    02 02 2021
Name: Date, Length: 1009, dtype: object

In [ ]:
df['formatted_distance'] = df['mrt_distance'].apply(lambda d: f'{round(d, 2):,}')

In [356]:
df_map_test['Date'].iloc[0]

Timestamp('2014-11-29 00:00:00')

In [178]:
#ents = [docs[i][0].ents]
i=10
docs[i][0].ents

(NPD Pankow,
 NPD,
 Buch,
 deutschen,
 Buch,
 NPD,
 Wagenburg,
 Karow,
 Punker_innen und Linke,
 NPD,
 Deutschen)

### Test pipe to find locations in df_lgbt['Story'] and df_lgbt['Header']
For comparisson, before including headers: <br>
len(stories_with_locs), len(stories_with_orgs), len(stories_with_just_orgs) = (891, 263, 32)

In [141]:
nlp = spacy.load('de_core_news_md')

In [142]:
story_indices = df_lgbt.index
story_districts = df_lgbt['District']

In [143]:
# list of strings from header and story, concatenated
titled_stories = [title + '. ' + story for (title, story) in zip(df_lgbt['Header'], df_lgbt['Story'])]

In [151]:
docs_data = [(ts, {'index': i, 'district': j}) for (ts, i, j) in zip(titled_stories, story_indices, story_districts)]

In [152]:
docs = list(nlp.pipe(docs_data, as_tuples=True))

**Pickle docs**

In [269]:
# pickle
with open('pickles/spacy_docs.pickle', 'wb') as f:
    pickle.dump(docs, f)

In [268]:
docs

Beschimpfungen und Böller gegen Demonstration. Passant*innen und Anwohner*innen beschimpfen Teilnehmende einer Demonstration, die sich gegen Homophobie richtet. Außerdem wird der Demonstrationszug mit einem Böller beworfen. Ein Teilnehmerin klagt im Anschluss über Schwindel.

**Sort found entities**

In [260]:
# Adds entities as metadata to doc_dict 
def entity_getter():
    '''
    Adds entities found in docs to a list in context dictionary
    '''
    no_ents = []
    for doc, doc_dict in docs:
        if len(doc.ents) == 0:
            no_ents.append(doc._.index)
        else:
            for ent in doc.ents:
                try:
                    doc_dict[ent.label_].append(ent.text)
                except:
                    doc_dict[ent.label_] = [ent.text]

In [24]:
for doc, doc_dict in docs:
    doc_dict['LOC'] = []
    doc_dict['ORG'] = []
    doc_dict['OTHER'] = []
    for ent in doc.ents:
        try: # key exists, append
            doc_dict[ent.label_].append(ent.text)
        

        
        else: #create key
        
        
        
        doct_dict[ent.label_]
        if ent.label_ == 'LOC':
            doc_dict['LOC'].append(ent.text)
        elif ent.label_ == 'ORG':
            doc_dict['ORG'].append(ent.text)
        else:
            doc_dict['OTHER'].append(ent.text)

In [43]:
docs[2]

(LGBQT feindliche Schmiererei entdeckt!. Am Abend gegen 20.10 Uhr stellen Polizeibeamte eine Farbschmiererei an dem Denkmal für die im Nationalsozialismus verfolgten Homosexuellen fest.,
 {'index': 1861, 'LOC': [], 'ORG': [], 'OTHER': [LGBQT]})

In [244]:
# list of indices of docs with no found entities
no_ents_index = [doc[1]['index'] for doc in docs if len(doc[0].ents) == 0]
no_ents_index

[8857, 16598, 18204, 19927, 20279, 21822]

In [249]:
# list of docs with no found entities
no_ents_docs = [doc[0] for doc in docs if doc[1]['index'] in no_ents_index]
no_ents_docs

[Paar aus homophober Motivation angespuckt. Am 04.07.17 gegen 17.00 Uhr wurden zwei Männer, die Hand in Hand auf der Rheinstraße liefen, aus einer Gruppe heraus angespuckt.,
 Rassistisches Mobbing und Kündigung. Eine Schwarze queere Person wurde auf der Arbeit im Rahmen einer Antidiskriminierungsfortbildung gemobbt und danach gekündigt.,
 LGBTIQ*feindliches Mobbing am Arbeitsplatz. Am Arbeitsplatz wurde ein trans Mensch gemobbt. Das Datum entspricht nicht dem Datum des Vorfalls, sondern dem Datum der Meldung.,
 E-Mail mit rassistischen und LGBTIQ*-feindlichem Inhalt. An das Bündnis #unteilbar wird eine beleidigende E-Mail mit rassistischen und LGBTIQ*-feindlichem Inhalt gesendet.,
 Frau am Mehringplatz aus LGBTIQ*-Feindlichkeit beleidigt und geschlagen. Um 21 Uhr soll eine 28-jährige Transfrau am Mehringplatz von einem bisher unbekannten Tatverdächtigen beleidigt und bespuckt worden sein. Im weiten Verlauf soll der Angreifer der Frau gegen den Kopf geschlagen haben. Ein Zeuge, der die 

**Add custom attributes 'index' and 'district' to Doc**

In [24]:
Doc.set_extension('index', default=None)
for doc, context in docs:
    doc._.index = context["index"]
docs[0][0]._.index

1707

In [25]:
Doc.set_extension('district', default=None)
for doc, context in docs:
    doc._.district = context["district"]
docs[0][0]._.district

'Tempelhof-Schöneberg'

**Test create entity**

In [662]:
docs[0][1]['locations'].append(docs[0][0].ents)

In [505]:
lgbt_stories = df_lgbt['Story']
docs = list(nlp.pipe(df_lgbt['Story']))

In [450]:
locations = []
organizations = []
for i, doc in enumerate(docs):
    locs_in_story = {i:[]}
    orgs_in_story = {i:[]}
    try:
        for ent in doc.ents:
            if ent.label_ == 'LOC':
                locs_in_story[i].append(ent)
            if ent.label_ == 'ORG':
                orgs_in_story[i].append(ent)
        if len(locs_in_story[i]) > 0:
            locations.append(locs_in_story)
        if len(orgs_in_story[i]) > 0:
            organizations.append(orgs_in_story)
    except:
        continue

In [451]:
stories_with_locs = []
stories_with_orgs = []
for i in locations:
    stories_with_locs.append(list(i.keys())[0])
for i in organizations:
    stories_with_orgs.append(list(i.keys())[0])

In [453]:
stories_with_just_orgs = [i for i in stories_with_orgs if i not in stories_with_locs]

In [455]:
stories_with_some_info = list(stories_with_locs)

In [456]:
for i in stories_with_just_orgs:
    stories_with_some_info.append(i)

In [457]:
# stories with no info: 82
len(stories) - len(stories_with_some_info)

86

In [454]:
len(stories_with_locs), len(stories_with_orgs), len(stories_with_just_orgs)

(891, 263, 32)

In [460]:
stories_with_no_info = [i for i in range(len(stories)) if i not in stories_with_some_info]

In [483]:
# Why didn't it recognize Adalbertstraße ?
[stories[i] for i in stories_with_no_info][6]

'Ein 31-jähriger Mann wird gegen 0.20 Uhr in der Adalbertstraße aus homophober Motivation ins Gesicht geschlagen.'

In [494]:
organizations

[{3: [Tennis Borussia Berlin,
   BSV Eintracht Mahlsdorf,
   Tennis Borussia Berlin,
   Eintracht Mahlsdorf,
   NPD,
   Tennis Borussia]},
 {4: [VfB Stuttgart,
   Hertha BSC Berlin,
   Hertha,
   KSC,
   Hertha BSC Berlin,
   Karlsruher Sport-Club,
   KSC,
   Hertha,
   KSC]},
 {9: [NSU]},
 {10: [NPD, NPD, Punker_innen und Linke, NPD]},
 {15: [NPD, Österreichischen Roten Kreuzes]},
 {26: [Bundeskanzleramt]},
 {29: [NPD Pankow]},
 {32: [Brüsseler Lobbyorganisation, European Dignity Watch]},
 {38: [Anti-Homosexuellen-Gesetze]},
 {44: [Axel-Springer-Verlag]},
 {45: [Junge Alternative, Flüchtlingskrise, Partei, Junge Alternative]},
 {49: [Berliner Polizei]},
 {51: [Aktion Lebensrecht für Alle]},
 {54: [Junge Alternative, Partei, Junge Alternative]},
 {55: [Polizeiliche Staatschutz]},
 {56: [Tagesspiegel]},
 {60: [Männern]},
 {62: [Bündnisses gegen Homophobie]},
 {63: [Kleinstpartei, Pro Deutschland, Gender-Wahn]},
 {67: [Junge Alternative, Partei, Junge Alternative]},
 {68: [Berliner Poliz

____

# Pickle df_lgbt

In [252]:
# pickle
with open('pickles/df_lgbt.pickle', 'wb') as f:
    pickle.dump(df_lgbt, f)

# Pickle df_map

In [254]:
# pickle df_map_test
with open('pickles/df_map_test.pickle', 'wb' ) as f:
    pickle.dump(df_map, f)